In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Advanced Lane Line Tracking

Due to errors with Jupyter notebook crashing when too many images are displayed, some visualization code has been commented out.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pdb
from pathlib import Path
from moviepy.editor import VideoFileClip

## Camera Calibration Step

In [ ]:
PATH = Path('./camera_calibration/')
n_cols = 9
n_rows = 6
objpoints = [] # 3D points in real world space
imgpoints = [] # 2D points in image plane

objp = np.zeros((n_cols*n_rows,3), np.float32)
objp[:,:2] = np.mgrid[0:n_cols,0:n_rows].T.reshape(-1,2)

In [ ]:
for filename in list(PATH.iterdir()):
    img = plt.imread(filename)
    if img is None:
        print('ERROR')
        pdb.set_trace()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (n_cols, n_rows), None)
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
        img = cv2.drawChessboardCorners(img, (n_cols, n_rows), corners, ret)
    else: 
        print('Unable to calibrate', filename)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

Let's grab a warped looking file and run it through the calibration step...

In [ ]:
# filename = list(PATH.iterdir())[11]
# img = plt.imread(filename)
# dst = cv2.undistort(img, mtx, dist, None, mtx)

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
# ax1.imshow(img)
# ax1.set_title('Original Image', fontsize=50)
# ax2.imshow(dst)
# ax2.set_title('Undistorted Image', fontsize=50)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def calibrate_image(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

## Let's take a look at some of the test images

This code is very handy for displaying a few images easily

In [ ]:
# PATH = Path('./test_images/')
# n_images = len(list(PATH.iterdir()))
# row_len = 4

# fig,axes=plt.subplots(n_images//row_len, row_len, figsize=(15,5))
# for i, ax in enumerate(axes.flat):
#     filename = list(PATH.iterdir())[i]
#     img = plt.imread(filename)
#     img = calibrate_image(img)
#     if img is None:
#         print('ERROR')
#         pdb.set_trace()
#     ax.imshow(img)
#     ax.set_title(str(filename).split('\\')[-1])
# plt.tight_layout()

# Sobel Gradient

In [ ]:
def abs_sobel_threshold(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Using mpimg.imread
    # Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
       sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
       sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Take the absolute value of the gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    max_sobel = np.max(abs_sobel)
    scaled_sobel = abs_sobel * 255 / max_sobel
    scaled_sobel = scaled_sobel.astype(np.uint8)
    # 5) Create a mask of 1's where the scaled gradient magnitude
            # is > thresh_min and < thresh_max
    s_binary = np.zeros_like(scaled_sobel)
    s_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
#     print(np.max(scaled_sobel))
    return s_binary

def mag_threshold(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Using mpimg.imread
    # Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel) # x gradient
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel) # y gradient

    # Calculate the magnitude 
    sum_sq = np.sqrt(sobel_x**2 + sobel_y**2)
    max_sobel = np.max(sum_sq)
    
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = sum_sq * 255 / max_sobel
    scaled_sobel = scaled_sobel.astype(np.uint8)
    
    # Create a binary mask where mag thresholds are met
    post_threshold = np.zeros_like(scaled_sobel)
    post_threshold[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return post_threshold

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Using mpimg.imread
    # Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel) # x gradient
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel) # y gradient
    # Take the absolute value of the x and y gradients
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)
    # Use arctan2 to calculate the direction of the gradient 
    atan = np.arctan2(abs_sobel_y, abs_sobel_x) # Element-wise arc tangent of y/x
    # Create a binary mask where direction thresholds are met
    post_threshold = np.zeros_like(atan)
    post_threshold[(atan >= thresh[0]) & (atan <= thresh[1])] = 1
    return post_threshold

In [ ]:
### Parameters to tune
ksize = 7 # Choose a larger odd number to smooth gradient measurements
thresh = (50, 255)
mag_thresh = (50, 255)
dir_thresh = (0.8, 1.2)

def gradient_sobel(img):
    ''' Takes an RGB image and computes several gradients to remove non-lane pixels
    '''
    # Apply each of the thresholding functions
    gradx = abs_sobel_threshold(img, orient='x', sobel_kernel=ksize, thresh=thresh)
    grady = abs_sobel_threshold(img, orient='y', sobel_kernel=ksize, thresh=thresh)
    mag_binary = mag_threshold(img, sobel_kernel=ksize, mag_thresh=mag_thresh)
    dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=dir_thresh)
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    return combined

## Let's test the functions we've developed:

It's clearly missing some areas (especially yellow lines), but not bad!

In [ ]:
# PATH = Path('./test_images/')
# n_images = len(list(PATH.iterdir()))
# row_len = 2
# image_width = 80/row_len
# image = None

# fig,axes=plt.subplots(n_images//row_len, row_len, figsize=(image_width,image_width/1.2))
# for i, ax in enumerate(axes.flat):
#     filename = list(PATH.iterdir())[i]
#     img = plt.imread(filename)
#     if img is None:
#         print('ERROR')
#         pdb.set_trace()
        
#     img = calibrate_image(img)
#     img = gradient_sobel(img)

#     ax.imshow(img, cmap='gray')
#     ax.set_title(str(filename).split('\\')[-1], fontsize=25)
# plt.tight_layout()

# HSL / HSV Color Space

In [ ]:
### Parameters to tune
s_min = 200
s_max = 255
v_min = 215
v_max = 255

def color_transform(img):
    ''' Takes an RGB image, converts it to HSV and HLS, and uses thresholds to remove non-lane pixels
    '''
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > s_min) & (s_channel < s_max)] = 1
    
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel > v_min) & (v_channel < v_max)] = 1
    
    out = np.zeros_like(s_channel)
    out[(s_binary == 1) | (v_binary == 1)] = 1
    return out

In [ ]:
# PATH = Path('./test_images/')
# n_images = len(list(PATH.iterdir()))
# row_len = 2
# image_width = 80/row_len
# image = None

# fig,axes=plt.subplots(n_images//row_len, row_len, figsize=(image_width,image_width/1.2))
# for i, ax in enumerate(axes.flat):
#     filename = list(PATH.iterdir())[i]
#     img = plt.imread(filename)
#     if img is None:
#         print('ERROR')
#         pdb.set_trace()
        
#     img = calibrate_image(img)
#     img = color_transform(img)

#     ax.imshow(img, cmap='gray')
#     ax.set_title(str(filename).split('\\')[-1], fontsize=25)
# plt.tight_layout()

# Test Gradient + Color Transform together

In [ ]:
def gradient_plus_color_transform(img):
    ''' Converts an RGB image to a bimary lane line image using gradients + color space transforms
    '''
    grad = gradient_sobel(img)
    color = color_transform(img)
    
    img = np.zeros_like(color)
    img[(grad == 1) | (color == 1)] = 255 # At the end of the pipeline, create an image that's 0-255 scaled
    return img

In [ ]:
# PATH = Path('./test_images/')
# n_images = len(list(PATH.iterdir()))
# row_len = 2
# image_width = 80/row_len
# image = None

# fig, axes = plt.subplots(n_images//row_len, row_len, figsize=(image_width,image_width/1.2))
# for i, ax in enumerate(axes.flat):
#     filename = list(PATH.iterdir())[i]
#     img = plt.imread(filename)
#     if img is None:
#         print('ERROR')
#         pdb.set_trace()
        
#     img = calibrate_image(img)
#     img = gradient_plus_color_transform(img)

#     ax.imshow(img, cmap='gray')
#     ax.set_title(str(filename).split('\\')[-1], fontsize=25)
# plt.tight_layout()

# Perspective Transform

In [ ]:
img = mpimg.imread('./test_images/test1.jpg')
# img = mpimg.imread('./test_images/test_lanes.png')
# img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Using mpimg.imread
img = calibrate_image(img)
img = gradient_plus_color_transform(img)
w = 1280
h = 720

fig, ax = plt.subplots(figsize=(40, 20))
ax.imshow(img, cmap='gray') 

offset = w*.25
dst = np.float32(
[[w*.25,0], # Top left
[w*.75, 0], # Top right
[w*.25, h], # Bottom left
[w*.75, h]]) # Bottom right

mid_width = 0.08
bot_width = 0.76
height_pct = .62
bottom_trim = 0.935

top_left =  (w*(0.5-mid_width/2), h*height_pct)
top_right = (w*(0.5+mid_width/2), h*height_pct)
bottom_left =  (w*(0.5-bot_width/2), h*bottom_trim)
bottom_right = (w*(0.5+bot_width/2), h*bottom_trim)

src = np.float32([top_left, top_right, bottom_left, bottom_right])
for pt in src:
    plt.plot(*pt,'bo',markersize=20)
    
for pt in dst:
    plt.plot(*pt,'ro',markersize=20)

In [ ]:
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# warped = cv2.warpPerspective(img, M, (w, h), flags=cv2.INTER_LINEAR)
# fig, ax = plt.subplots(figsize=(40, 20))
# ax.imshow(warped, cmap='gray') 

In [ ]:
# PATH = Path('./test_images/')
# n_images = len(list(PATH.iterdir()))
# row_len = 2
# plot_width = 80/row_len
# plot_height = plot_width/1.2
# image = None

# fig, axes = plt.subplots(n_images//row_len, row_len, figsize=(plot_width,plot_height))
# for i, ax in enumerate(axes.flat):
#     filename = list(PATH.iterdir())[i]
#     img = plt.imread(filename)
#     if img is None:
#         print('ERROR')
#         pdb.set_trace()
        
#     img = calibrate_image(img)
#     img = gradient_plus_color_transform(img)
#     img = cv2.warpPerspective(img, M, img.shape[::-1], flags=cv2.INTER_LINEAR)

#     ax.imshow(img, cmap='gray')
#     ax.set_title(str(filename).split('\\')[-1], fontsize=25)
# plt.tight_layout()

# Line Tracker

In [ ]:
xm_per_pix=3.7/500 # Lane is 3.7 meters wide, and 500 pixels wide from bird's eye view
ym_per_pix=30/720 # Visible lane is about 30 meters long, and 720 pixels high from bird's eye view 

def graph_lane_lines(binary_warped):
    ''' Takes in a binary birds eye image and returns the camera's
    perspective green polygon. You must separately overlay the original image
    '''
    histogram = np.sum(binary_warped[h//2:,:], axis=0)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    midpoint = w//2
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    window_height = h//nwindows
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0]) # Split up the X and Y coords
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each lane line
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, h-1, num=h) # yvals = range(0,h)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))

    real_out_img = np.zeros_like(out_img)
    cv2.fillPoly(real_out_img, np.int_([left_line_pts]), (0, 0, 255))
    camera_perspective = cv2.warpPerspective(real_out_img, Minv, (w, h), flags=cv2.INTER_LINEAR)

    ### Determine the real world lane curvature radius
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)   
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    avg_curve = np.mean([left_curverad, right_curverad])
    
    ### Add text to image regarding position and lane curvature
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center - w/2) * xm_per_pix
    side_pos = 'left' if center_diff > 0 else 'right'
    cv2.putText(camera_perspective, 'The Radius of Curvature = ' + str(round(avg_curve,3))+' (m) ',
        (50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(camera_perspective, 'The Vehicle is ' + str(abs(round(center_diff,3)))+'m ' + side_pos + ' of center',
        (50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    
    ### Add a bird's eye view on the top right corner of the image with lane line markers
    small_birds_eye = np.zeros((h,w,3))
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-25, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+25, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-25, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+25, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    cv2.fillPoly(small_birds_eye, np.int_([left_line_pts]), (255,255,255))
    cv2.fillPoly(small_birds_eye, np.int_([right_line_pts]),(255,255,255))
    small_birds_eye = cv2.resize(small_birds_eye, (0,0), fx=0.3, fy=0.3)
    camera_perspective[0:small_birds_eye.shape[0], 1280-small_birds_eye.shape[1]:] = small_birds_eye 
    
    return camera_perspective

In [ ]:
def process_image(img):
    calibrated = calibrate_image(img)
    binary = gradient_plus_color_transform(calibrated)
    binary_warped = cv2.warpPerspective(binary, M, (w,h), flags=cv2.INTER_LINEAR).astype(np.uint8) 
    polygon = graph_lane_lines(binary_warped)
    result = cv2.addWeighted(polygon, 0.5, img, 1, 0)
    plt.imsave('final.png',result)
    return result

In [ ]:
output_file = 'output_images/video.mp4'
input_clip = VideoFileClip('project_video.mp4').subclip(0,1)
# input_clip = VideoFileClip("project_video.mp4")
output_clip = input_clip.fl_image(process_image) # NOTE: this function expects color images
%time output_clip.write_videofile(output_file, audio=False)

input_clip.reader.close()
input_clip.audio.reader.close_proc()

output_clip.reader.close()
output_clip.audio.reader.close_proc()

del input_clip
del output_clip